In [53]:
%load_ext autoreload
%autoreload 2
#Assessments with confirmed active results
#107617, 107614

# 17 mins for current operation with all ids
# 2 mins for 100 ids. 

#To DO
#Implement logging to the whole operations

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
from airflow import DAG
from airflow.operators.python import PythonOperator
import logging
import os

os.makedirs('logs', exist_ok=True)
logging.basicConfig(filename='./logs/Illuminate_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Illuminate Operations Logging Instance')


access_token, expires_in = get_access_token()

#For testing
assessment_id_list = [115519, 115900]
# temp = get_all_assessments_metadata(access_token)
# assessment_id_list = list(temp['assessment_id'].unique())
# missing_ids_from_metadata = ['114845'] # Add assessments that are not present in assessements metadata
# assessment_id_list = add_missing_assessments(assessment_id_list, missing_ids_from_metadata)

test_results_group, log_results_group = loop_through_assessment_scores(access_token, assessment_id_list, 'Group') #Seperate table
test_results_standard, log_results_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'Standard')
test_results_no_standard, log_results_no_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'No_Standard')

test_results_combined = bring_together_test_results(test_results_no_standard, test_results_standard)
test_results_view = create_test_results_view(test_results_combined, '23-24') #add in grade level col, string matching


# access_token, expires_in = get_access_token()



#Write dfs to SFTP
#Takes 4 mins with current 167 test_ids. 

#takes 25 mins to run all 

#41000 rows to now 58000 rows

#Student_Rosters.txt is coming through at 6:20
#Ask to get that moved up

# also, i was wondering if there's a way we can rename the units so i can re-order these bars:
# shelby wants it to be like unit 1, unit 2, unit 3, interim 1, unit 4, unit 5, interim 2, etc. etc.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The length of the ID_list is 2
The length of the ID_list is 2
The length of the ID_list is 2


In [ ]:
also, i'm noting that for assignment 115519, not all of the standards are importing into the db
for example, there are 10 standards tested in this assignment + the overall score


In [6]:
import pandas as pd

v.loc[v['assessment_id'] == 115519]['standard_code'].unique()


array(['CCSS.ELA-Literacy.RI.3.2', 'CCSS.ELA-Literacy.RI.3.4',
       'CCSS.ELA-Literacy.RI.3.8', 'CCSS.ELA-Literacy.L.3.5',
       'CCSS.ELA-Literacy.RL.3.1', 'CCSS.ELA-Literacy.RI.3.1', 'percent'],
      dtype=object)

In [9]:
c.loc[c['assessment_id'] == 115519]['standard_code'].unique()

array(['CCSS.ELA-Literacy.RI.3.2', 'CCSS.ELA-Literacy.RI.3.4',
       'CCSS.ELA-Literacy.RI.3.8', 'CCSS.ELA-Literacy.L.3.5',
       'CCSS.ELA-Literacy.RL.3.1', 'CCSS.ELA-Literacy.RI.3.1', 'percent'],
      dtype=object)

In [32]:
%load_ext autoreload
%autoreload 2
from modules.assessments_endpoints import *
from modules.auth import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
from airflow import DAG
from airflow.operators.python import PythonOperator
import logging
import os

access_token, expires_in = get_access_token()
s, log = get_assessment_scores(access_token, 115519, 'Standard')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[2024-11-14T15:55:51.618-0600] {auth.py:63} INFO - Calling API token endpoint
[2024-11-14T15:55:51.623-0600] {auth.py:69} INFO - Succesfully retrieved API token
[2024-11-14T15:55:51.627-0600] {assessments_endpoints.py:125} INFO - https://icefps.illuminateed.com/live/rest_server.php/Api/AssessmentAggregateStudentResponsesStandard/?page=1&assessment_id=115519&limit=1000&date_taken_start=2024-07-01&date_taken_end=2024-11-14
[2024-11-14T15:55:52.033-0600] {assessments_endpoints.py:128} INFO - The status code for assessment_id 115519 is 200
[2024-11-14T15:55:52.044-0600] {assessments_endpoints.py:135} INFO - Here is the num of pages for 115519 id - 2 pages
[2024-11-14T15:55:52.046-0600] {assessments_endpoints.py:144} INFO - Results are present for _id 115519, num_results 1710, page 1
[2024-11-14T15:55:52.071-0600] {assessments_endpoints.py:179} INFO - Here is the url_ext for the next page AssessmentAggre

In [34]:
s['standard_code'].unique()

array(['CCSS.ELA-Literacy.RI.3.2', 'CCSS.ELA-Literacy.RI.3.4',
       'CCSS.ELA-Literacy.RI.3.8', 'CCSS.ELA-Literacy.L.3.5',
       'CCSS.ELA-Literacy.RL.3.1', 'CCSS.ELA-Literacy.RI.3.1'],
      dtype=object)

In [ ]:
headers = {
    "Authorization": f"Bearer {access_token}"
}

page=1
_id=115519
current_date = '2024-11-14'

url_ext = f'AssessmentAggregateStudentResponsesStandard/?page={page}&assessment_id={_id}&limit=1000&date_taken_start=2024-07-01&date_taken_end={current_date}'

# logging.info(base_url_illuminate + url_ext)
response = requests.get(base_url_illuminate + url_ext, headers=headers)

r = response.status_code

pd.DataFrame(json.loads(response.content)['results'])['standard_code'].unique() #710 rows. Only 1 page

# 'CCSS.ELA-Literacy.RL.3.1', 'CCSS.ELA-Literacy.RL.3.2',
#        'CCSS.ELA-Literacy.RL.3.3', 'CCSS.ELA-Literacy.RL.3.4',
#        'CCSS.ELA-Literacy.RL.3.6'


#page 2
# 'CCSS.ELA-Literacy.L.3.5', 'CCSS.ELA-Literacy.RI.3.1',
#        'CCSS.ELA-Literacy.RI.3.2', 'CCSS.ELA-Literacy.RI.3.4',
#        'CCSS.ELA-Literacy.RI.3.8', 'CCSS.ELA-Literacy.RL.3.1'

array(['CCSS.ELA-Literacy.L.3.5', 'CCSS.ELA-Literacy.RI.3.1',
       'CCSS.ELA-Literacy.RI.3.2', 'CCSS.ELA-Literacy.RI.3.4',
       'CCSS.ELA-Literacy.RI.3.8', 'CCSS.ELA-Literacy.RL.3.1'],
      dtype=object)

In [ ]:
o, log = get_assessment_scores(access_token, 115519, 'Standard') #why does this display 2000 results. When the log is saying 1710

[2024-11-14T16:14:13.245-0600] {assessments_endpoints.py:125} INFO - https://icefps.illuminateed.com/live/rest_server.php/Api/AssessmentAggregateStudentResponsesStandard/?page=1&assessment_id=115519&limit=1000&date_taken_start=2024-07-01&date_taken_end=2024-11-14
[2024-11-14T16:14:13.458-0600] {assessments_endpoints.py:128} INFO - The status code for assessment_id 115519 is 200
[2024-11-14T16:14:13.466-0600] {assessments_endpoints.py:135} INFO - Here is the num of pages for 115519 id - 2 pages
[2024-11-14T16:14:13.468-0600] {assessments_endpoints.py:144} INFO - Results are present for _id 115519, num_results 1710, page 1
[2024-11-14T16:14:13.483-0600] {assessments_endpoints.py:179} INFO - Here is the url_ext for the next page AssessmentAggregateStudentResponsesStandard/?page=1&assessment_id=115519&limit=1000&date_taken_start=2024-07-01&date_taken_end=2024-11-14
[2024-11-14T16:14:13.485-0600] {assessments_endpoints.py:125} INFO - https://icefps.illuminateed.com/live/rest_server.php/Api/

In [ ]:
'CCSS.ELA-Literacy.RL.3.1', 
'CCSS.ELA-Literacy.RL.3.2',
'CCSS.ELA-Literacy.RL.3.3', 
'CCSS.ELA-Literacy.RL.3.4',
'CCSS.ELA-Literacy.L.3.5',
'CCSS.ELA-Literacy.RL.3.6',
'CCSS.ELA-Literacy.L.3.5', 
'CCSS.ELA-Literacy.RI.3.1',
'CCSS.ELA-Literacy.RI.3.2', 
'CCSS.ELA-Literacy.RI.3.4',
'CCSS.ELA-Literacy.RI.3.8', 
'CCSS.ELA-Literacy.RL.3.1'

{'page': 1, 'num_pages': 0, 'num_results': 0, 'results': []}

In [2]:
import pandas as pd
c =pd.read_csv('/home/g2015samtaylor/illuminate/assessment_results_combined.csv')
v =pd.read_csv('/home/g2015samtaylor/illuminate/assessment_results_view.csv')
sr =pd.read_table('/home/icef/powerschool/Student Rosters.txt')

/tmp/ipykernel_1216677/2925511554.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  c =pd.read_csv('/home/g2015samtaylor/illuminate/assessment_results_combined.csv')


In [62]:
v.loc[v['grade'].isna()]['local_student_id'].to_csv('misssing_ids.csv', index=False)

In [61]:
sr.loc[sr['STUDENTS.Student_Number'] == 602409]

,STUDENTS.ID,STUDENTS.dcid,STUDENTS.SchoolID,STUDENTS.State_StudentNumber,STUDENTS.Student_Number,STUDENTS.Grade_Level,STUDENTS.Enroll_Status,S_CA_STU_X.primarydisability,S_CA_STU_X.elastatus


In [64]:
pd.read_csv('/home/g2015samtaylor/airflow/git_directory/Illuminate/misssing_ids.csv')

,STUDENTS.Student_Number
0,602333
1,602409
2,602580
3,602409
4,602333
...,...
2841,602605
2842,602605
2843,602460
2844,602435


In [68]:
sr.loc[sr['STUDENTS.Student_Number'] == 602409]

,STUDENTS.ID,STUDENTS.dcid,STUDENTS.SchoolID,STUDENTS.State_StudentNumber,STUDENTS.Student_Number,STUDENTS.Grade_Level,STUDENTS.Enroll_Status,S_CA_STU_X.primarydisability,S_CA_STU_X.elastatus


In [189]:
%load_ext autoreload
%autoreload 2
#Assessments with confirmed active results
#107617, 107614

# 17 mins for current operation with all ids
# 2 mins for 100 ids. 

#To DO
#Implement logging to the whole operations

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
from airflow import DAG
from airflow.operators.python import PythonOperator
import logging
import os

access_token, expires_in = get_access_token()
test_results_view = create_test_results_view(test_results_combined, '23-24') #add in grade level col, string matching

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# just noticed that assessment id 114845 (4th grade IM unit 2 assessment) is missing from the data
#This should be solved once ran again in airflow. 
#Double check this

# Some assessments missing grade level (ex. Assessment 115900)
# Another idea is to do a join with the Powerschool demographic data using student ID and pulling grade from there

#Done
# Algebra II interim exam (141449) says “Algebra” as curriculum
# This seems to be a common thing for Algebra I and Algebra II. Should say Algebra I or Algebra II
# Pre Calculus missing curriculum data (141457) or incorrect (141501)
# For these ones on curriculum we want this to come across as Pre-Calc or Pre-Calculus
# Stats (141460) needs to be “Statistics” not “IM”
# Biology (141506) curriculum data incorrect
# Biology for the curriculum here
# 115520, 115519, 115538 says IM when it should be Into Reading
# Some assessments missing “Unit” (IA/Interim in title should say “Interim)


# TO DO

Changes for the test_results_combined frame
Now represents all assessments standard & no standard. 

Both frames brought together, and added in standard_code column. 
All values in 'standard_code' column with a percent value are no_standard values in the 'Standard_No_Standard' column as well. So if you were looking to fitler on this. Looks like you can do it either way

Changes for the test_results_view
asssesment_id column is now present
values in kinder column that were 'K' are now 0

General notes
The date taken filter on all of these assessment pulls is from 2024-07-01 up to the current run day
